In [12]:
%pip install keyboard pyAudio

   ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
   -------------------- ------------------ 30.7/58.1 kB 660.6 kB/s eta 0:00:01
   --------------------------------------- 58.1/58.1 kB 771.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyaudio
import numpy as np
import wave
from scipy.signal import butter, lfilter
import keyboard

In [3]:
# Constants
CHUNK = 1024  # Number of audio samples per frame
FORMAT = pyaudio.paInt16  # Audio format
CHANNELS = 1  # Mono audio
RATE = 44100  # Sampling rate (Hz)
LATENCY_THRESHOLD = 100  # Max latency (ms)
FILE_NAME = "output_cleaned.wav"  # Output file name

# Butterworth filter for noise reduction
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

audio = pyaudio.PyAudio() # Initialize PyAudio
stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, output=True, frames_per_buffer=CHUNK)  # Open audio stream

print("Recording and processing in real-time... Press 'p' to pause, 'r' to resume, and Pause the Code to stop.")

wf = wave.open(FILE_NAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)

is_paused = False 
try:
    while True:
        if keyboard.is_pressed('p'):
            if not is_paused:
                print("Pausing...")
                is_paused = True  
        elif keyboard.is_pressed('r'):
            if is_paused:
                print("Resuming...")
                is_paused = False  
        if not is_paused:
            raw_data = stream.read(CHUNK, exception_on_overflow=False)
            np_data = np.frombuffer(raw_data, dtype=np.int16)

            cleaned_data = bandpass_filter(np_data, lowcut=300, highcut=3400, fs=RATE, order=6)
            output_data = cleaned_data.astype(np.int16).tobytes()
            stream.write(output_data)
            wf.writeframes(output_data)

except KeyboardInterrupt:
    print("\nStopped by user (Ctrl+C).")

finally:
    stream.stop_stream()
    stream.close()
    audio.terminate()
    wf.close()
    print(f"Processed audio saved as {FILE_NAME}")

Recording and processing in real-time... Press 'p' to pause, 'r' to resume, and Ctrl+C to stop.

Stopped by user (Ctrl+C).
Processed audio saved as output_cleaned.wav
